In [25]:
publish.html("""
Cloud Formation:
<select id='action'>
 <option value="create" selected>create</option>
 <option value="remove">remove</option>
 <option value="suspend">suspend</option>
 <option value="resume">resume</option>
</select>
<button id="run">Run</button>
<br><div id='output'></div>
""")

Cloud Formation:
 
 create 
 remove 
 suspend 
 resume 
 
 Run

In [67]:
publish.javascript("""
  window.now = function() { 
     var time = new Date(); 
     return time.getHours()+":"+time.getMinutes()+":"+time.getSeconds() 
  }
  $("#run").off("click")
  $("#run").on("click", function(data) {
  var data = "val action=\""+$("#action").val()+"\"\nval debug=false"
  IPython.notebook.get_cell(2).code_mirror.setValue(data) 
  IPython.notebook.select(2)
  IPython.notebook.execute_cells_below()
})
""")
def message(s:String) = publish.javascript("$('#output').text(now()+' "+s+"')")


defined function message

In [186]:
val action="create"
val debug=false

action: String = "create"
debug: Boolean = false

In [187]:
// options
import ammonite.ops._

val stackName = "test"
/// parameters
// template is for us-east-1
// change images for different regions
val region = "us-east-1"
val imageId = "ami-10f53a06"
val instanceType = "t2.small"
val keyName = "atomic"
val template = pwd/up/'aws/"kubernetes.yml"

import ammonite.ops._


stackName: String = "test"
region: String = "us-east-1"
imageId: String = "ami-10f53a06"
instanceType: String = "t2.small"
keyName: String = "atomic"
template: Path = /Users/msciab/Dropbox/Work/MacBookPro/MicheleSciabarra/Mosaico/cloud/aws/kubernetes.yml

In [188]:
import $ivy.`com.amazonaws:aws-java-sdk:1.11.22`

import $ivy.$                                   

In [189]:
import com.amazonaws.services.cloudformation.model._
import collection.JavaConverters._

val create  = action  == "create"
val remove  = action  == "remove"
val suspend = action  == "suspend"
val resume  = action  == "resume"

import com.amazonaws.services.cloudformation.model._

import collection.JavaConverters._


create: Boolean = true
remove: Boolean = false
suspend: Boolean = false
resume: Boolean = false

In [190]:
import com.amazonaws.services._
import com.amazonaws.services.cloudformation._

val cf = new AmazonCloudFormationClient() 
def status = {
    val stack = cf.describeStacks
        .getStacks.asScala
        .filter( _.getStackName == stackName).headOption
    stack.map(_.getStackStatus)
}
def statusLoop {
    val st = status
    message(st.getOrElse("-"))
    if(st.nonEmpty && st.get.endsWith("_IN_PROGRESS")) {
        Thread.sleep(1000)
        statusLoop
    }
}

import com.amazonaws.services._

import com.amazonaws.services.cloudformation._


cf: AmazonCloudFormationClient = com.amazonaws.services.cloudformation.AmazonCloudFormationClient@818851d
defined function status
defined function statusLoop

## Creation

In [191]:
// read template
import scala.io._
import com.amazonaws.services.cloudformation.model._

val cf = new cloudformation.AmazonCloudFormationClient

// create
def createRequest() = {
    import com.amazonaws.services.cloudformation._
    import com.amazonaws.services.cloudformation.model._
    val cfr = new CreateStackRequest()
    cfr.setStackName(stackName)
    val params =  Seq(
      new Parameter().withParameterKey("KeyName").withParameterValue(keyName),
      new Parameter().withParameterKey("InstanceType").withParameterValue(instanceType),
      new Parameter().withParameterKey("ImageId").withParameterValue(imageId)    
    ) 
    cfr.setParameters(params.asJava)
    cfr
}

if(create) {
    val cfr = createRequest()
    val body = Source.fromFile(template.toString).getLines.mkString("\n")
    cfr.setTemplateBody(body)
    // run request
    val res = cf.createStack(cfr)
    statusLoop
}

import scala.io._

import com.amazonaws.services.cloudformation.model._


cf: AmazonCloudFormationClient = com.amazonaws.services.cloudformation.AmazonCloudFormationClient@27278a3d
defined function createRequest

## Cleanup

In [192]:
// remove
def removeRequest() = 
    new DeleteStackRequest().withStackName(stackName)

if(remove) { 
    cf.deleteStack(removeRequest())
    statusLoop
}

//removeRequest()

defined function removeRequest

## Building the inventory

In [193]:
import com.amazonaws.services.ec2._
import com.amazonaws.regions.Region

val ec2 = new AmazonEC2Client()
val regions = Regions.fromName(region)
ec2.setRegion(Region.getRegion(regions))

import com.amazonaws.services.ec2._

import com.amazonaws.regions.Region


ec2: AmazonEC2Client = com.amazonaws.services.ec2.AmazonEC2Client@6a441a44
regions: Regions = US_EAST_1

In [210]:
import collection.JavaConverters._

val instances = for {
 reservation <- ec2.describeInstances.getReservations.asScala
 instance <- reservation.getInstances.asScala
} yield {
    instance
}
val ids = instances.filter(_.getState.getName == "running").map(_.getInstanceId)
val stopped = instances.filter(_.getState.getName == "stopped").map(_.getInstanceId)



import collection.JavaConverters._


instances: collection.mutable.Buffer[Instance] = ArrayBuffer(
  {InstanceId: i-0b61965d913df17ad,ImageId: ami-10f53a06,State: {Code: 16,Name: running},PrivateDnsName: ip-10-0-0-55.ec2.internal,PublicDnsName: ,StateTransitionReason: ,KeyName: atomic,AmiLaunchIndex: 0,ProductCodes: [],InstanceType: t2.small,LaunchTime: Fri Apr 14 14:34:33 CEST 2017,Placement: {AvailabilityZone: us-east-1d,GroupName: ,Tenancy: default,},Monitoring: {State: disabled},SubnetId: subnet-bde1b1d8,VpcId: vpc-f57b7893,PrivateIpAddress: 10.0.0.55,PublicIpAddress: 34.201.193.95,Architecture: x86_64,RootDeviceType: ebs,RootDeviceName: /dev/sda1,BlockDeviceMappings: [{DeviceName: /dev/sda1,Ebs: {VolumeId: vol-05af7d9eb3b487c9d,Status: attached,AttachTime: Fri Apr 14 14:34:34 CEST 2017,DeleteOnTermination: false}}],VirtualizationType: hvm,ClientToken: test-Master-DSDT14T074QI,Tags: [{Key: Application,Value: arn:aws:cloudformation:us-east-1:794403675119:stack/t...
ids: collection.m

In [202]:
import com.amazonaws.services.ec2.model._
import collection.JavaConverters._

val susp = if(suspend) {
    Some(ec2.stopInstances(new StopInstancesRequest(ids.asJava)))
    message("SUSPENDING")
} else None

val resm = if(resume) {
     Some(ec2.startInstances(new StartInstancesRequest(stopped.asJava)))
     message("RESUMING")
} else None

import com.amazonaws.services.ec2.model._

import collection.JavaConverters._


susp: Any = None
resm: Any = None

In [211]:
import collection.JavaConverters._

//val reservation = ec2.describeInstances.getReservations.asScala
if(resume || create) {
    val ips = for {
     reservation <- ec2.describeInstances.getReservations.asScala
     instance <- reservation.getInstances.asScala
     if instance.getState.getName == "running"
    } yield {
        instance.getPrivateIpAddress -> instance.getPublicIpAddress
    }

    val inventory = Seq(
            "[all:vars]",
            s"kube_master_host=${ips.head._2}",
            s"kube_master=${ips.head._1}",    
            "[aws]"
        ) ++ ips.map(_._2) ++
          Seq("[masters]", s"${ips.head._2} local_ip=${ips.head._1}") ++
          Seq("[nodes]") ++
          ips.tail.map(t => s"${t._2} local_ip=${t._1}")

    val inv = pwd/up/'aws/"inventory"
    rm! inv
    write(inv, inventory.mkString("\n"))
    show(inventory)
}

List(
  "[all:vars]",
  "kube_master_host=34.201.193.95",
  "kube_master=10.0.0.55",
  "[aws]",
  "34.201.193.95",
  "34.201.18.231",
  "34.201.7.210",
  "[masters]",
  "34.201.193.95 local_ip=10.0.0.55",
  "[nodes]",
  "34.201.18.231 local_ip=10.0.0.106",
  "34.201.7.210 local_ip=10.0.0.56"
)


import collection.JavaConverters._

//val reservation = ec2.describeInstances.getReservations.asScala


In [213]:
if(resume || create) {
     val ips = for {
     reservation <- ec2.describeInstances.getReservations.asScala
     instance <- reservation.getInstances.asScala
     if instance.getState.getName == "running"
    } yield {
        instance.getPrivateIpAddress -> instance.getPublicIpAddress
    }
    val kubecfg = s"""
    apiVersion: v1
    clusters:
    - cluster:
        server: http://${ips.head._2}:8080
      name: aws
    contexts:
    - context:
        cluster: aws
        user: ""
      name: aws
    current-context: aws
    """
    println(kubecfg)
    val cfg = pwd/up/'aws/"kubeconfig"
    rm! cfg
    write(cfg, kubecfg)
}


    apiVersion: v1
    clusters:
    - cluster:
        server: http://34.201.193.95:8080
      name: aws
    contexts:
    - context:
        cluster: aws
        user: ""
      name: aws
    current-context: aws
    


res207: collection.mutable.Buffer[(String, String)] = ArrayBuffer(
  ("10.0.0.102", "34.195.120.75"),
  ("10.0.0.42", "54.89.149.59"),
  ("10.0.0.219", "54.92.185.245")
)